In [39]:
import pandas as pd
import os
import numpy as np

In [33]:
os.chdir("/Users/kevinkinsey/Developer/Agnirath/TrackTesting")
os.getcwd()

'/Users/kevinkinsey/Developer/Agnirath/TrackTesting'

In [ ]:
data = pd.read_csv("./data/raw/raw_solar&wind.csv", )
data

,ghi,azimuth,zenith,wind_speed_10m,wind_direction_10m,period_end,period
0,0,47,127,6.9,192,2025-06-13T16:05:00Z,PT5M
1,0,46,128,6.9,193,2025-06-13T16:10:00Z,PT5M
2,0,45,129,7.0,194,2025-06-13T16:15:00Z,PT5M
3,0,44,130,7.0,195,2025-06-13T16:20:00Z,PT5M
4,0,43,131,7.1,195,2025-06-13T16:25:00Z,PT5M
...,...,...,...,...,...,...,...
4028,0,50,123,4.4,204,2025-06-27T15:45:00Z,PT5M
4029,0,50,124,4.4,204,2025-06-27T15:50:00Z,PT5M
4030,0,49,125,4.4,205,2025-06-27T15:55:00Z,PT5M
4031,0,48,126,4.4,206,2025-06-27T16:00:00Z,PT5M


In [35]:
data.dtypes

ghi                     int64
azimuth                 int64
zenith                  int64
wind_speed_10m        float64
wind_direction_10m      int64
period_end             object
period                 object
dtype: object

In [36]:
data['period_end'] = pd.to_datetime(data['period_end'], utc=True, format='mixed').dt.tz_convert('Asia/Kolkata')

### Clean Up

In [37]:
# Define your desired UTC window
start_time = pd.Timestamp('2025-06-15 08:00:00', tz='Asia/Kolkata')
end_time   = pd.Timestamp('2025-06-15 18:00:00', tz='Asia/Kolkata')

# Filter the DataFrame within that window
filtered_data = data[
    (data['period_end'] >= start_time) & (data['period_end'] <= end_time)
]
filtered_data = filtered_data.reset_index(drop=True)
filtered_data

,ghi,azimuth,zenith,wind_speed_10m,wind_direction_10m,period_end,period
0,301,-71,60,7.6,240,2025-06-15 08:00:00+05:30,PT5M
1,317,-71,59,7.7,240,2025-06-15 08:05:00+05:30,PT5M
2,332,-71,58,7.7,240,2025-06-15 08:10:00+05:30,PT5M
3,347,-71,57,7.7,241,2025-06-15 08:15:00+05:30,PT5M
4,363,-71,56,7.7,241,2025-06-15 08:20:00+05:30,PT5M
...,...,...,...,...,...,...,...
116,89,69,78,6.6,209,2025-06-15 17:40:00+05:30,PT5M
117,76,68,79,6.6,208,2025-06-15 17:45:00+05:30,PT5M
118,64,68,80,6.6,208,2025-06-15 17:50:00+05:30,PT5M
119,52,68,81,6.6,207,2025-06-15 17:55:00+05:30,PT5M


In [31]:
(18 - 8) * 60/5

120.0

**NOTE:** assumign flat panel, you dont have to calc azimuth and all; its jsut ghi

In [40]:
final_data = filtered_data.copy()
final_data['wind_speed'] = final_data['wind_speed_10m']
final_data['wind_direction'] = np.deg2rad(final_data['wind_direction_10m'])
final_data['time'] = final_data['period_end'].dt.time

final_data = final_data[['time', 'ghi', 'wind_speed', 'wind_direction']]
final_data

,time,ghi,wind_speed,wind_direction
0,08:00:00,301,7.6,4.188790
1,08:05:00,317,7.7,4.188790
2,08:10:00,332,7.7,4.188790
3,08:15:00,347,7.7,4.206243
4,08:20:00,363,7.7,4.206243
...,...,...,...,...
116,17:40:00,89,6.6,3.647738
117,17:45:00,76,6.6,3.630285
118,17:50:00,64,6.6,3.630285
119,17:55:00,52,6.6,3.612832


In [42]:
final_data.to_csv("./data/sol&wind.csv", index=False)